# Aula 47 - Projeto Cury de entregas - Streamlit Visão Empresa

## Problema de negócio

A Cury Company é uma empresa de tecnologia que criou um aplicativo que conecta
restaurantes, entregadores e pessoas.<br><br>
Através desse aplicativo, é possível realizar o pedido de uma refeição, em qualquer
restaurante cadastrado, e recebê-lo no conforto da sua casa por um entregador
também cadastrado no aplicativo da Cury Company.<br><br>
A empresa realiza negócios entre restaurantes, entregadores e pessoas, e gera
muitos dados sobre entregas, tipos de pedidos, condições climáticas, avaliação dos
entregadores e etc. Apesar da entrega estar crescento, em termos de entregas, o
CEO não tem visibilidade completa dos KPIs de crescimento da empresa.<br><br>
Você foi contratado como um Cientista de Dados para criar soluções de dados para
entrega, mas antes de treinar algoritmos, a necessidade da empresa é ter um os
principais KPIs estratégicos organizados em uma única ferramenta, para que o CEO
possa consultar e conseguir tomar decisões simples, porém importantes.<br><br>
A Cury Company possui um modelo de negócio chamado Marketplace, que fazer o
intermédio do negócio entre três clientes principais:<br> Restaurantes;<br> entregadores;<br> e
pessoas compradoras.

## 1. Planejamento do projeto e limpeza do dataset

### A visão: Crescimento

### Do lado da empresa:

Iremos criar um dashboard da visão da empresa no Streamlit

## <u>2. Execução do projeto - Streamlit

### <u>2.1. Data extract

#### 2.1.1. Requerimentos

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import plotly.express as px
from matplotlib import pyplot as plt
import folium
from haversine import haversine #biblioteca para o cálculo das distâncias entre latitudes e longitudes

#### 2.1.2. Helper functions

In [3]:
#Transformação de casas decimais das notações científicas 
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#Dimensionamento do dataset
def dimensionamento(dataframe):
    print(f"O dataset tem {dataframe.shape[0]} linhas.")
    print(f"O dataset tem {dataframe.shape[1]} colunas.")
    return None

#Tratamento do espaçamento no dados do tipo string - sem loop for
def space_clean(dataframe, coluna):
    dataframe.loc[:, coluna] = dataframe.loc[:, coluna].str.strip()

#### 2.1.3. Data load

In [4]:
data = pd.read_csv("../Datasets/train.csv")

In [5]:
data.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.75,75.89,22.77,75.91,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.91,77.68,13.04,77.81,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.91,77.68,12.92,77.69,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.00,76.98,11.05,77.03,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.97,80.25,13.01,80.29,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


### <u>2.2. Data describe

#### 2.2.1. Data dimensions

In [6]:
dimensionamento(data)

O dataset tem 45593 linhas.
O dataset tem 20 colunas.


In [7]:
data.shape

(45593, 20)

#### 2.2.2. Limpeza dos dados

##### 2.2.2.1. Análise e alteração dos tipos de dados

##### 2.2.2.2. Tratamento de espaçamento em dados do tipo string

##### 2.2.2.3. Manipulação e tratamento de dados faltantes

In [149]:
#Modificação dos tipos dos dados
#Feature Delivery_person_Age
#data["Delivery_person_Age"] = data["Delivery_person_Age"].astype(int)

#Resumo do problema: Não é possível, nesse momento, realizar a alteração do tipo de string para int, já que existem
#valores nulos em linhas da feature. Por isso, nossa opção, nesse momento, é tratar primeiramente os valores nulos e após
#isso alterar o tipo da coluna.

#Levantamento de valores nulos:
#Como vimos, abaixo não temlos qualquer valor nulo, mas quando tentamos acima alterar o tipo da coluna e tivemos como retorn
#que não é possível a alteração de tipo a linhas cujo valor é nulo, tomamos por base que os valores nulos reais das linhas,
#pelo menos na feature Delivery_person_Age, contem espaçamento. Assim, vamos buscar as linhas considerando um possível espaçamento
#e, caso encontremos, iremos tratar esse espaçamento, para depois tratar o valor nulo.
data.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weatherconditions              0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken(min)                0
dtype: int64

In [150]:
#Avaliando a existência de espaçamentos na coluna "Delivery_person_Age".
##Existem valores nulos com espaçamento e, como vimos abaixo, não é apenas essa variável que possui esse tipo de problema. 
#Assim sendo, iremos tratar todos esses espaçamentos.

data.loc[data["Delivery_person_Age"] == "NaN ", :].head()

##Nesse momento vimos que a coluna Weatherconditions possui valores nulos com status "conditions NaN". Vamos tratar também,
#mas substituindo o que tiver esse status para um status 'NaN ', com espaçamento, para criarmos um padrão a ser ajustado
#em outro momento. Esse tratamento específico será na próxima célula.

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
42,0x4f0,MUMRES17DEL02,NaN,NaN,19.12,72.91,19.20,72.99,06-04-2022,NaN,18:35:00,conditions Cloudy,Medium,1,Drinks,scooter,1,No,Metropolitian,(min) 33
74,0xa9f,BANGRES13DEL01,NaN,NaN,12.94,77.61,12.98,77.65,11-03-2022,NaN,15:30:00,conditions NaN,NaN,1,Drinks,scooter,0,No,Metropolitian,(min) 19
120,0x6e2,VADRES06DEL01,NaN,NaN,22.31,73.17,22.42,73.28,02-04-2022,NaN,18:25:00,conditions NaN,NaN,3,Snack,electric_scooter,0,No,Metropolitian,(min) 25
140,0x1ec7,PUNERES05DEL03,NaN,NaN,18.54,73.91,18.57,73.94,09-03-2022,NaN,23:40:00,conditions Stormy,Low,2,Snack,scooter,0,No,NaN,(min) 19
179,0x1b58,JAPRES03DEL01,NaN,NaN,26.91,75.80,26.98,75.87,12-03-2022,NaN,17:50:00,conditions Sandstorms,Medium,0,Drinks,motorcycle,0,No,Metropolitian,(min) 43


In [151]:
#Tratamento dos valores nulos da feature Delivery_person_Age, Delivery_person_Ratings, Time_Orderd, Road_traffic_density e City
#Existem valores nulos com espaçamento e, como vimos abaixo, não é apenas essa variável que possui esse tipo de problema. 
#Assim sendo, iremos tratar todos esses espaçamentos.
space_clean(data, "Delivery_person_Age")
space_clean(data, "Delivery_person_Ratings")
space_clean(data, "Time_Orderd")
space_clean(data, "Road_traffic_density")
space_clean(data, "multiple_deliveries")
space_clean(data, "City")
space_clean(data, "Type_of_vehicle")
space_clean(data, "ID")
space_clean(data, "Festival")

In [152]:
#Testando se os espaçamentos foram retirados - Delivery_person_Age
data.loc[data["Delivery_person_Age"] == "NaN ", :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [153]:
#Testando se os espaçamentos foram retirados - Delivery_person_Ratings
data.loc[data["Delivery_person_Ratings"] == "NaN ", :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [154]:
#Testando se os espaçamentos foram retirados - Time_Orderd
data.loc[data["Time_Orderd"] == "NaN ", :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [155]:
#Testando se os espaçamentos foram retirados - Road_traffic_density
data.loc[data["Road_traffic_density"] == "NaN ", :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [156]:
#Testando se os espaçamentos foram retirados - City
data.loc[data["City"] == "NaN ", :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [157]:
#Substituição do valor 'conditions NaN' por apenas 'NaN' - variável Weatherconditions
#Vamos usar a linha de número 74 para o teste de replace
data.loc[74, "Weatherconditions"] = data.loc[74, "Weatherconditions"].replace('conditions NaN', 'NaN')

In [158]:
#Testando se o valor foi substituído
data.loc[74, "Weatherconditions"]

'NaN'

In [159]:
#Agora vamos aplicar o replace a todas as linhas com essa condição
for i in range( len(data) ):
    if data.loc[i, "Weatherconditions"] == 'conditions NaN':
        data.loc[i, "Weatherconditions"] = data.loc[i, "Weatherconditions"].replace('conditions NaN', 'NaN')

In [160]:
#Vamos testar agora abaixo se os valores foram trocados:
data.loc[data["Weatherconditions"] == 'conditions NaN', :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [161]:
#Agora vamos avaliar novamente se tudo deu certo e se temos enfim os valores nulos correto, usando, por exemplo, a  variável 
#Delivery_person_Age
data.loc[data["Delivery_person_Age"] == "NaN", "Delivery_person_Age"]

42       NaN
74       NaN
120      NaN
140      NaN
179      NaN
        ... 
45508    NaN
45518    NaN
45519    NaN
45526    NaN
45566    NaN
Name: Delivery_person_Age, Length: 1854, dtype: object

In [162]:
#Nesse momento vamos iniciar o processo de conversão das variáveis
data.head(1)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.75,75.89,22.77,75.91,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24


In [163]:
data.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [164]:
#Variável - critério: excluir os valores nulos da base
#Após a média, preencher esses valores nulos com essa média
#Excluindo NaN das bases
#Selecionando apenas linhas não nulas
df = data.copy()
df = df.loc[df["Delivery_person_Age"] != "NaN", :]
#Convertendo o tipo
df["Delivery_person_Age"] = np.array(df["Delivery_person_Age"], dtype= np.int64)
#Convertendo o tipo
df["Delivery_person_Ratings"] = df["Delivery_person_Ratings"].astype(float)
#Convertendo o tipo, criando a feature Week_of_year e a convertendo para int
df["Order_Date"] = pd.to_datetime(df["Order_Date"], format="%d-%m-%Y")
#Selecionando apenas linhas não nulas
df = df.loc[df["multiple_deliveries"] != "NaN", :]
#Convertendo o tipo
df["multiple_deliveries"] = np.array(df["multiple_deliveries"], dtype= np.int64)

In [165]:
df.dtypes

ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int64
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                     int64
Festival                               object
City                                   object
Time_taken(min)                        object
dtype: object

In [166]:
#Agora vamos adotar o critério de média das features inteiras para preencher as linhas nulas
print(df["Delivery_person_Age"].mean().round(0))
print(df["multiple_deliveries"].mean().round(0))
print(df["Delivery_person_Ratings"].mean().round(2))
print("")
#Vamos adotar o critério do maior resultado, que é "Condition Fog"
print("{}%".format((df["Weatherconditions"].value_counts()/42805*100)))

30.0
1.0
4.63

conditions Fog          17.03
conditions Stormy       16.88
conditions Cloudy       16.67
conditions Sandstorms   16.55
conditions Windy        16.54
conditions Sunny        16.13
NaN                      0.21
Name: Weatherconditions, dtype: float64%


In [167]:
#vamos substituir os valores com os resultados acima levantados
for i in range( len(data) ):
    if data.loc[i, "Delivery_person_Age"] == 'NaN':
        data.loc[i, "Delivery_person_Age"] = data.loc[i, "Delivery_person_Age"].replace("NaN", "30")

for i in range( len(data) ):
    if data.loc[i, "Delivery_person_Ratings"] == 'NaN':
        data.loc[i, "Delivery_person_Ratings"] = data.loc[i, "Delivery_person_Ratings"].replace("NaN", "4.63")

for i in range( len(data) ):
    if data.loc[i, "multiple_deliveries"] == 'NaN':
        data.loc[i, "multiple_deliveries"] = data.loc[i, "multiple_deliveries"].replace("NaN", "1")
        
for i in range( len(data) ):
    if data.loc[i, "Weatherconditions"] == 'NaN':
        data.loc[i, "Weatherconditions"] = data.loc[i, "Weatherconditions"].replace("NaN", "conditions Fog")

for i in range( len(data) ):
    if data.loc[i, "City"] == "NaN":
        data.loc[i, "City"] = data.loc[i, "City"].replace("NaN", "Semi-Urban")

for i in range( len(data) ):
    if data.loc[i, "Road_traffic_density"] == "NaN":
        data.loc[i, "Road_traffic_density"] = data.loc[i, "Road_traffic_density"].replace("NaN", "Low")

In [168]:
#Testando se os valores foram substituidos

data.loc[data["Delivery_person_Age"] == "NaN", :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [169]:
data.loc[data["multiple_deliveries"] == "NaN", :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [170]:
data.loc[data["Weatherconditions"] == "NaN", :]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)


In [171]:
data.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [172]:
#Convertendo o tipo
data["Delivery_person_Age"] = np.array(data["Delivery_person_Age"], dtype= np.int64)
#Convertendo o tipo
data["Delivery_person_Ratings"] = data["Delivery_person_Ratings"].astype(float)
#Convertendo o tipo, criando a feature Week_of_year e a convertendo para int
data["Order_Date"] = pd.to_datetime(data["Order_Date"], format="%d-%m-%Y")
data['Week_of_year'] = data['Order_Date'].dt.strftime('%U')
data['Week_of_year'] = np.array(data["Week_of_year"], dtype= np.int64)
#Convertendo o tipo
data["multiple_deliveries"] = np.array(data["multiple_deliveries"], dtype= np.int64)        

In [173]:
data.dtypes

ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int64
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                     int64
Festival                               object
City                                   object
Time_taken(min)                        object
Week_of_year                            int64
dtype: object

In [174]:
#Por último vamos fazer o tratamento de texto em números na feature Time_taken(min)
data["Time_taken(min)"]

0        (min) 24
1        (min) 33
2        (min) 26
3        (min) 21
4        (min) 30
           ...   
45588    (min) 32
45589    (min) 36
45590    (min) 16
45591    (min) 26
45592    (min) 36
Name: Time_taken(min), Length: 45593, dtype: object

In [175]:
data.reset_index(drop=True)
for i in range( len(data)):
    data.loc[i, "Time_taken(min)"] = re.findall(r'\d+', data.loc[i, 'Time_taken(min)'])

In [176]:
data["Time_taken(min)"]

0        [24]
1        [33]
2        [26]
3        [21]
4        [30]
         ... 
45588    [32]
45589    [36]
45590    [16]
45591    [26]
45592    [36]
Name: Time_taken(min), Length: 45593, dtype: object

In [180]:
data["Time_taken(min)"] = data["Time_taken(min)"].astype(str)

# retirando o []
for i in range(len(data)): 
    data.loc[i, 'Time_taken(min)'] = str(data.loc[i,'Time_taken(min)']).strip('[]')

# retirando as ""
for i in range(len(data)): 
    data.loc[i, 'Time_taken(min)'] = str(data.loc[i,'Time_taken(min)']).strip("''")

data['Time_taken(min)'] = np.array(data['Time_taken(min)'], dtype= np.int64)
data.dtypes

ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int64
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                     int64
Festival                               object
City                                   object
Time_taken(min)                         int64
Week_of_year                            int64
dtype: object

#### Extração do Dataset tratado para um arquivo .csv

In [181]:
data.to_csv('Datasets/train_tratado.csv', index=False)